In [1]:
import numpy as np
import pandas as pd

In [2]:
food = pd.read_csv("food.csv")
nutrient = pd.read_csv("new_nutrient.csv")
food_nutrient = pd.read_csv("food_nutrient.csv")

In [3]:
food

,fdc_id,data_type,description,food_category_id,publication_date
0,167512,sr_legacy_food,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01
1,167513,sr_legacy_food,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01
2,167514,sr_legacy_food,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01
3,167515,sr_legacy_food,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01
4,167516,sr_legacy_food,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01
...,...,...,...,...,...
7788,175300,sr_legacy_food,"Game meat, buffalo, water, cooked, roasted",17,2019-04-01
7789,175301,sr_legacy_food,"Game meat, elk, raw",17,2019-04-01
7790,175302,sr_legacy_food,"Game meat, elk, cooked, roasted",17,2019-04-01
7791,175303,sr_legacy_food,"Game meat, goat, raw",17,2019-04-01


In [4]:
nutrient

,id,name,unit_name
0,1008,Energy,KCAL
1,1004,Total lipid (fat),G
2,1258,"Fatty acids, total saturated",G
3,1257,"Fatty acids, total trans",G
4,1003,Protein,G
5,1149,"Salt, NaCl",MG
6,1079,"Fiber, total dietary",G
7,1005,"Carbohydrate, by difference",G
8,2000,"Sugars, Total",G
9,1253,Cholesterol,MG


In [5]:
print(nutrient["id"].values)

[1008 1004 1258 1257 1003 1149 1079 1005 2000 1253]


In [6]:
print(food_nutrient.shape)
food_nutrient = food_nutrient[food_nutrient["nutrient_id"].isin(nutrient["id"].values)]
print(food_nutrient.shape)

(644125, 11)
(63433, 11)


In [7]:
food_nutrient = food_nutrient.drop(["id", "data_points", "min_year_acquired", "footnote", "median", "min", "max", "derivation_id"], axis=1)

In [8]:
transposed = food_nutrient[food_nutrient["fdc_id"] == 167516].transpose().replace(nutrient["id"].values, nutrient["name"].values)
transposed

,62,86,94,120,145,152,167,168
fdc_id,167516.0,167516.0,167516.0,167516.0,167516.0,167516.0,167516.0,167516.0
nutrient_id,"Fatty acids, total saturated",Protein,"Fiber, total dietary",Cholesterol,Energy,"Sugars, Total",Total lipid (fat),"Carbohydrate, by difference"
amount,1.898,6.58,2.2,15.0,273.0,4.3,9.22,41.05


In [9]:
food_nutrient["nutrient_id"] = food_nutrient["nutrient_id"].replace(nutrient["id"].values, nutrient["name"].values)
food_nutrient

,fdc_id,nutrient_id,amount
0,167512,Protein,5.880
3,167512,"Fiber, total dietary",1.200
6,167512,Cholesterol,0.000
7,167512,"Fatty acids, total trans",4.412
8,167512,"Fatty acids, total saturated",2.941
...,...,...,...
644093,175304,"Carbohydrate, by difference",0.000
644094,175304,Protein,27.100
644096,175304,"Fiber, total dietary",0.000
644107,175304,Cholesterol,75.000


In [10]:
food = food.drop(["data_type", "food_category_id", "publication_date"], axis=1)
food

,fdc_id,description
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar..."
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger..."
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa..."
3,167515,"George Weston Bakeries, Thomas English Muffins"
4,167516,"Waffles, buttermilk, frozen, ready-to-heat"
...,...,...
7788,175300,"Game meat, buffalo, water, cooked, roasted"
7789,175301,"Game meat, elk, raw"
7790,175302,"Game meat, elk, cooked, roasted"
7791,175303,"Game meat, goat, raw"


In [11]:
food_nutrient["fdc_id"] = food_nutrient["fdc_id"].replace(food["fdc_id"].values, food["description"].values)

In [12]:
food_nutrient = food_nutrient.rename(columns={"fdc_id": "food", "nutrient_id": "nutrient"})

In [13]:
def convert_units(row):
    if nutrient[nutrient["name"] == "Cholesterol"]["unit_name"] == "MG":
        return row["amount"] / 1000
    else:
        return row["amount"]

In [14]:
food_nutrient[food_nutrient["nutrient"].isin(["Salt, NaCl", "Cholesterol"])]["amount"] / 1000.0

6         0.000
23        0.000
120       0.015
239       0.013
315       0.016
          ...  
643868    0.061
643880    0.055
643975    0.073
644008    0.057
644107    0.075
Name: amount, Length: 7394, dtype: float64

In [15]:
adjusted_values = food_nutrient[food_nutrient["nutrient"].isin(["Salt, NaCl", "Cholesterol"])]["amount"] / 1000.0

In [16]:
food_nutrient.loc[food_nutrient["nutrient"].isin(["Salt, NaCl", "Cholesterol"]), "amount"] = adjusted_values
food_nutrient.loc[food_nutrient["nutrient"].isin(["Salt, NaCl", "Cholesterol"]), "amount"]

6         0.000
23        0.000
120       0.015
239       0.013
315       0.016
          ...  
643868    0.061
643880    0.055
643975    0.073
644008    0.057
644107    0.075
Name: amount, Length: 7394, dtype: float64

In [17]:
food_nutrient["nutrient"] = food_nutrient["nutrient"].replace(
    nutrient["name"].values,
    [
    "calories",
    "fat_g",
    "saturates_g",
    "trans_fat_g",
    "protein_g",
    "salt_g",
    "fiber_g",
    "carbohydrates_g",
    "sugars_g",
    "cholesterol_g",
    ]
)

In [18]:
food_nutrient

,food,nutrient,amount
0,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",protein_g,5.880
3,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",fiber_g,1.200
6,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",cholesterol_g,0.000
7,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",trans_fat_g,4.412
8,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",saturates_g,2.941
...,...,...,...
644093,"Game meat, goat, cooked, roasted",carbohydrates_g,0.000
644094,"Game meat, goat, cooked, roasted",protein_g,27.100
644096,"Game meat, goat, cooked, roasted",fiber_g,0.000
644107,"Game meat, goat, cooked, roasted",cholesterol_g,0.075


In [19]:
food_list = []

for food_name in food_nutrient["food"].unique():
    macros_list = food_nutrient[food_nutrient["food"] == food_name].drop("food", axis=1).to_dict("records")
    final_macros = {}

    for macro in macros_list:
        final_macros[macro["nutrient"]] = macro["amount"]

    food_dict = {
        "name": food_name,
        "macros": final_macros
    }
    
    food_list.append(food_dict)

print(len(food_list))

7793


In [20]:
import json

json_string = json.dumps(food_list, indent=4)
with open('food.json', 'w') as f:
    f.write(json_string)